In [1]:
import pandas as pd
import torch as torch
from neuralforecast.auto import AutoRNN, AutoLSTM



In [2]:
from neuralforecast.tsdataset import TimeSeriesDataset

df = pd.read_csv('../data/SBER_250914_250915.csv', sep=';')

df["date_str"]=df["<DATE>"].apply(lambda x: str(x).zfill(6)).astype(str)
df["time_str"]=df["<TIME>"].apply(lambda x: str(x).zfill(6)).astype(str)
df["datetime_str"]=df["date_str"]+df["time_str"]
df["ds"]= pd.to_datetime(df["datetime_str"], format='%y%m%d%H%M%S', utc=True)
df["unique_id"] = "SBER"

df.drop(["date_str", "time_str", "datetime_str", "<DATE>", "<TIME>", "<OPEN>", "<HIGH>", "<LOW>", "<VOL>"], axis=1, inplace=True)
df.rename(columns={"<CLOSE>": "y"}, inplace=True)

df = df.tail(200)

dataset, *_ = TimeSeriesDataset.from_df(df)

In [3]:
df

,y,ds,unique_id
829,302.25,2025-09-15 11:48:00+00:00,SBER
830,302.26,2025-09-15 11:49:00+00:00,SBER
831,302.28,2025-09-15 11:50:00+00:00,SBER
832,302.23,2025-09-15 11:51:00+00:00,SBER
833,302.47,2025-09-15 11:52:00+00:00,SBER
...,...,...,...
1024,302.36,2025-09-15 15:03:00+00:00,SBER
1025,302.37,2025-09-15 15:04:00+00:00,SBER
1026,302.34,2025-09-15 15:05:00+00:00,SBER
1027,302.44,2025-09-15 15:06:00+00:00,SBER


In [4]:
torch.set_float32_matmul_precision('high')
config = dict(max_steps=1, val_check_steps=1, input_size=5, encoder_hidden_size=8)
model = AutoLSTM(h=1, config=config, num_samples=1, verbose=False)

model.fit(dataset=dataset)
y_hat = model.predict(dataset=dataset)


(_train_tune pid=17856) C:\venvs\vertr312\Lib\site-packages\ray\tune\integration\pytorch_lightning.py:198: `ray.tune.integration.pytorch_lightning.TuneReportCallback` is deprecated. Use `ray.tune.integration.pytorch_lightning.TuneReportCheckpointCallback` instead.
(_train_tune pid=17856) Seed set to 1
(_train_tune pid=17856) GPU available: False, used: False
(_train_tune pid=17856) TPU available: False, using: 0 TPU cores
(_train_tune pid=17856) `Trainer(val_check_interval=1)` was configured so validation will run after every batch.
(_train_tune pid=17856) 
(_train_tune pid=17856)   | Name         | Type          | Params | Mode 
(_train_tune pid=17856) -------------------------------------------------------
(_train_tune pid=17856) 0 | loss         | MAE           | 0      | train
(_train_tune pid=17856) 1 | padder_train | ConstantPad1d | 0      | train
(_train_tune pid=17856) 2 | scaler       | TemporalNorm  | 0      | train
(_train_tune pid=17856) 3 | hist_encoder | LSTM          | 9

Epoch 0: 100%|██████████| 1/1 [00:00<00:00, 29.33it/s, v_num=0, train_loss_step=3.960]
Validation: |          | 0/? [00:00<?, ?it/s]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 0: 100%|██████████| 1/1 [00:00<00:00, 14.92it/s, v_num=0, train_loss_step=3.960, valid_loss=0.190, train_loss_epoch=3.960]


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=1` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
`Trainer(val_check_interval=1)` was configured so validation will run after every batch.


Predicting: |          | 0/? [00:00<?, ?it/s]

In [5]:
y_hat


array([[302.37036]], dtype=float32)